# Näitekood Riigi Teataja pealkirja otsinguks

Päringulaiendaja kasutamine võimaldab meil teha päringuid, mis leiavad ülesse otsiõna kõik vormid eeldusel, et päringulaiendaja sisendandmebaas on ajakohane. Seetõttu on mõistlik teha otsing kahe tasemeline:

* päringulaiendajat kasutav otsing (kiire)
* kõiki sõnavorme pimedalt genereeriv otsing (aeglane)

Mõlema otsingu tegemiseks on tarvis kasutada Riigi Teataja veebiteenust, mis lubab otsida pealkirjadest täpseid sõnu.  

In [201]:
import math
import requests

from io import BytesIO
from pandas import concat
from pandas import read_csv
from pandas import DataFrame
from datetime import date

from typing import List, Union

## I. Päringulaiendaja liidestamine

In [231]:
def generate_worldforms_with_query_extender(words: Union[List[str], str],
                                            case: bool = True,
                                            search_sublemmas: bool = False):
    """
    Uses web service to get all wordforms existing in the documents for a list of lemmas.
    By default the extender keeps track of in which case the characters are, e.g. Euroopa and euroopa are different. 
    Returns a two column table with columns input, input_lemma, extension_type, score and search_wordform.
    The score shows the likelihood of the search wordform. The higher the better. 

    If the flag search_sublemmas is set returns also compund words containing.
    if the flag case is false extension is done in case insensitive manner.
    """
    words = [words] if isinstance(words, str) else words
    if not all(map(lambda x: x.find('\t') == -1, words)):
        raise ValueError("Inputs cannot contain '\\t' character. It corrupts the output")

    ANALYSER = " https://smart-search.tartunlp.ai/api/ea_paring_sl/tsv"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {"params":{"otsi_liitsõnadest": search_sublemmas}, "tss": "\t".join(words)}
    
    response = requests.post(ANALYSER, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"

    return (read_csv(BytesIO(response.content), delimiter='\t')
            .rename(columns={
                'lemma': 'input_lemma', 
                'type': 'extension_type', 
                'confidence': 'score',
                'wordform': 'search_wordform'})
            [['input', 'input_lemma', 'extension_type', 'score', 'search_wordform']]
           )

### Standardnäited päringusõna laiendamisest

Vaikimisi laiendatakse otsisõna arvestades suurtäheortograafiat. Sealjuures võib astuste erinevus olla vaid õiges sisendlemmas samas kui laiendite hulk jääb samaks. 

In [232]:
display(generate_worldforms_with_query_extender('Euroopa'))
display(generate_worldforms_with_query_extender('euroopa'))
display(generate_worldforms_with_query_extender('EL'))
display(generate_worldforms_with_query_extender('el'))

,input,input_lemma,extension_type,score,search_wordform
0,Euroopa,Euroopa,word,820,Euroopa
1,Euroopa,Euroopa,word,819,euroopa


,input,input_lemma,extension_type,score,search_wordform
0,euroopa,Euroopa,word,820,Euroopa
1,euroopa,Euroopa,word,819,euroopa
2,euroopa,euroopa,word,819,euroopa


,input,input_lemma,extension_type,score,search_wordform
0,EL,EL,word,10,EL


,input,input_lemma,extension_type,score,search_wordform


Õige ja kirjavigase otsisõne väljund on sama kui vastavad lemmad langevad kokku. Samas võib juhtuda, et kirjaveaga vormi pole üldse sõnastikus.

In [222]:
display(generate_worldforms_with_query_extender('president'))
display(generate_worldforms_with_query_extender('bresident'))
display(generate_worldforms_with_query_extender('presidemt'))

,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president


,input,input_lemma,extension_type,score,search_wordform
0,bresident,president,suggestion,18,presidendi
1,bresident,president,suggestion,2,president
2,bresident,president,compound,18,presidendi
3,bresident,president,compound,2,president


,input,input_lemma,extension_type,score,search_wordform


Lipu `search_sublemmas` sättimine võimaldab leida ka liitsõnu, mis sisaldavad otsisõna alamlemmana. 

In [255]:
display(generate_worldforms_with_query_extender('president', search_sublemmas=True))
display(generate_worldforms_with_query_extender('tuuma', search_sublemmas=True))
display(generate_worldforms_with_query_extender('aastane', search_sublemmas=True))
display(generate_worldforms_with_query_extender('19', search_sublemmas=True))

,input,input_lemma,extension_type,score,search_wordform
0,president,president,word,18,presidendi
1,president,president,word,2,president
2,president,president,compound,1,asepresidendi


,input,input_lemma,extension_type,score,search_wordform
0,tuuma,tuum,word,18,tuuma
1,tuuma,tuum,compound,1,tuumaavarii
2,tuuma,tuum,compound,2,tuumaavariist
3,tuuma,tuum,compound,1,tuumakahjustuste
4,tuuma,tuum,compound,1,tuumakütuse
5,tuuma,tuum,compound,4,tuumamaterjali
6,tuuma,tuum,compound,1,tuumaohutus
7,tuuma,tuum,compound,1,tuumaohutus
8,tuuma,tuum,compound,1,tuumaohutuse
9,tuuma,tuum,compound,2,tuumarelva


,input,input_lemma,extension_type,score,search_wordform
0,aastane,aastane,word,7,aastane
1,aastane,aastane,word,13,aastase
2,aastane,aastane,word,1,aastasele
3,aastane,aastane,word,1,aastases
4,aastane,aastane,word,3,aastasse
5,aastane,aastane,word,5,aastast
6,aastane,aastane,word,6,aastaste
7,aastane,aastane,word,3,aastastele
8,aastane,aastane,compound,1,19aastaste
9,aastane,aastane,compound,1,19aastastele


,input,input_lemma,extension_type,score,search_wordform
0,19,19,word,4,19
1,19,19,compound,1,19aastaste
2,19,19,compound,1,19aastastele


Erikujuliste nimede korral leitakse ülesse ka kõikvõimalikud tekstides olevad nime käändevormid.

In [251]:
display(generate_worldforms_with_query_extender('Strasbourg'))
display(generate_worldforms_with_query_extender('d’Ivoire'))
display(generate_worldforms_with_query_extender('ÜRO'))
display(generate_worldforms_with_query_extender('Meri'))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,ÜRO,ÜRO,word,8,ÜRO


,input,input_lemma,extension_type,score,search_wordform
0,Meri,Meri,word,2,Meri
1,Meri,Meri,word,7,mere
2,Meri,Meri,word,1,mereni
3,Meri,Meri,word,1,meres
4,Meri,Meri,word,8,meri
5,Meri,Meri,word,1,merre


### Päringufraaside laiendamine

**Oluline:** Päringulaiendja ei tööta mitmesõnaliste pärisnimedega ja terminoloogia fraasidega. Seda eelkõige selle tõttu, et Riigi Teataja veebiteenus ei toeta fraaside otsimist. Teiseks on vastav fraaside normaliseerimismoodul Riigi Teataja kodulehel olemas. 


Fraaside laiendamise saaks lisada päringulaiendajasse, kuid see eeldaks fikseeritud fraaside tuvastamist tekstist. Selleks oleks vaja dokumentide lisaindekseerimisteenust, mis tuvastaks nimekirja alusel dokumendis olevaid fraase. Vastav laiendus eeldaks EstNLTK teegi kasutamist, mis on GPL v2 litsensiga. Me käsitleme seda eraldi osas [`01_dokumentide_indekseerimine`](../01_dokumentide_indekseerimine/). 

In [241]:
display(generate_worldforms_with_query_extender('Euroopa Liit'))
display(generate_worldforms_with_query_extender(['euroopa', 'liit']))

,input,input_lemma,extension_type,score,search_wordform


,input,input_lemma,extension_type,score,search_wordform
0,euroopa,Euroopa,word,820,Euroopa
1,euroopa,Euroopa,word,819,euroopa
2,euroopa,euroopa,word,819,euroopa
3,liit,liit,word,486,liidu
4,liit,liit,word,14,liiduga
5,liit,liit,word,15,liidule
6,liit,liit,word,17,liidult
7,liit,liit,word,7,liidus
8,liit,liit,word,1,liidust
9,liit,liit,word,36,liit


### Lühendite ja numbrite laiendamine

In [253]:
display(generate_worldforms_with_query_extender('19'))
display(generate_worldforms_with_query_extender('CO2'))
display(generate_worldforms_with_query_extender('COVID-19'))
display(generate_worldforms_with_query_extender('Valka-2'))
display(generate_worldforms_with_query_extender('TKNE-5'))






,input,input_lemma,extension_type,score,search_wordform
0,CO2,CO2,word,1,CO2


,input,input_lemma,extension_type,score,search_wordform
0,COVID-19,COVID-19,word,105,COVID-19


,input,input_lemma,extension_type,score,search_wordform
0,Valka-2,Valka-2,word,1,Valka-2


,input,input_lemma,extension_type,score,search_wordform
0,TKNE-5,TKNE-5,word,1,TKNE-5


,input,input_lemma,extension_type,score,search_wordform
0,19aastane,19aastane,word,1,19aastaste
1,19aastane,19aastane,word,1,19aastastele


## II. Riigi Teataja baasotsinguteenuse liidestamine 

In [156]:
def search_rt_caption(wordform:str, match_type:str='koik_sonad', **kwargs):
    """
    Potential match types are: 
    * autocomplete – leiab aktid, mille pealkirjas on suvalises järjekorras sõnad, mis algavad otsitud sõnadega
    * koik_sonad – leiab aktid, mille pealkirjas esinevad kõik otsitud sõnad suvalises järjekorras
    * täpne – leiab aktid, mille pealkirjas esineb otsitud fraas
    """
    
    SEARCH_QUERY = 'https://www.riigiteataja.ee/api/oigusakt_otsing/1/otsi'

    # Let us pick all documents
    ARG_STRUCTURE = \
    {
        'leht': (int, 1),
        'limiit': (int, 500),
        'kehtiv': (date, None),
        'tulemused': (bool, True),
        'kehtivKehtetus': (bool, False),
        'mitteJoustunud': (bool, False),
        'kov': (bool, True),
        'dokument': (str, 'seadus')
    }

    # Check that keyword arguments are correct and have right type 
    payload = kwargs.copy() 
    for key, value in payload.items():
        arg_type, default_value = ARG_STRUCTURE.get(key, (None, None))
        if arg_type is None:
            raise ValueError(f'Unknown argument: {key}')
        elif not isinstance(value, arg_type):
            raise ValueError(f'Argument {key} must be of type {arg_type}')

    # Update important payload arguments        
    payload['leht'] = 1
    payload['kehtiv'] = date.today()
    payload['pealkiri'] = wordform
    payload['pealkiriOtsinguTyyp'] = match_type
    payload['filter'] = True
    payload['grupeeri'] = False
    payload['limiit'] = payload.get('limiit', 500)
    payload['dokument'] = payload.get('dokument', 'seadus')
        
    response = requests.get(SEARCH_QUERY, params=payload)
    assert response.status_code== 200, 'GET request failed'
    response = response.json()

    # Get the number of responce pages
    assert 'aktid' in response, 'Missing payload'
    assert 'metaandmed' in response, 'Missing meta field'
    assert 'kokku' in response['metaandmed'], 'Missing meta field'
    assert 'limiit' in response['metaandmed'], 'Missing meta field'

    total_count = response['metaandmed']['kokku']
    document_limit = response['metaandmed']['limiit']
    max_page = math.ceil(total_count/document_limit)

    if total_count == 0:
        return DataFrame(columns=['document_type', 'document_title', 'commencement_date', 'global_id'])
    
    # Iterate over responce sheets
    query_results = [None] * max_page 
    for page in range(max_page):

        payload['leht'] = page + 1
        response = requests.get(SEARCH_QUERY, params=payload)
        assert response.status_code== 200, 'GET request failed'
        response = response.json()

        document_count = len(response['aktid'])
        query_results[page] = DataFrame({
            'document_type': [None] * document_count,             
            'document_title': [None] * document_count, 
            'commencement_date': [None] * document_count,
            'global_id': [None] * document_count})

        for i, document in enumerate(response['aktid']):
            query_results[page].loc[i, 'global_id'] = document['globaalID']
            query_results[page].loc[i, 'document_title'] = document['pealkiri']
            query_results[page].loc[i, 'document_type'] = document['liik']
            query_results[page].loc[i, 'commencement_date'] = document['kehtivus'].get('algus')
        
    return (concat(query_results, axis=0)
            .sort_values(['document_title', 'commencement_date'], ascending=[True, False])
            .reset_index(drop=True))

search_rt_caption('presidendi')

,document_type,document_title,commencement_date,global_id
0,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2014-01-01,129122012010
1,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2013-01-01,128122011068
2,seadus,Riigikogu ja Vabariigi Presidendi poolt nimeta...,2012-01-01,13262582
3,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,122062016013
4,seadus,Vabariigi Presidendi ametihüve seadus,2018-01-01,127062017010
5,seadus,Vabariigi Presidendi ametihüve seadus,2016-07-01,113122014059
6,seadus,Vabariigi Presidendi ametihüve seadus,2014-01-01,129122011152
7,seadus,Vabariigi Presidendi ametihüve seadus,2012-01-01,128122010026
8,seadus,Vabariigi Presidendi ja Vabariigi Valitsuse li...,2004-07-01,769953
9,seadus,Vabariigi Presidendi töökorra seadus,2017-07-07,127062017011


In [236]:
search_rt_caption('Euroopa Liitu')

,document_type,document_title,commencement_date,global_id


## III. Lihtne otsimisloogika

In [95]:
input_words = input("Riigi Teataja pealkirjaotsing:").split(' ')
print(f'Otsime dokumentidest sõnu: {input_words}')

Riigi Teataja pealkirjaotsing: presidendi ametiraha


Otsime dokumentidest sõnu: ['presidendi', 'ametiraha']


In [119]:
input_words = ['bresidendi', 'ametiraha']
input_words = [x.lower() for x in input_words]
extension_table = generate_worldforms_with_query_extender(input_words, search_sublemmas=False)
display(extension_table)

,input,input_lemma,extension_type,score,search_wordform
0,bresidendi,president,suggestion,18,presidendi
1,bresidendi,president,suggestion,2,president
2,bresidendi,president,compound,18,presidendi
3,bresidendi,president,compound,2,president


In [116]:
generate_worldforms_with_query_extender(['ametiraha'])

,input,input_lemma,extension_type,score,search_wordform


In [140]:
query_data = extension_table.groupby(['input', 'extension_type'], as_index=False).aggregate(search_words=('search_wordform', set))
display(query_data)

,input,extension_type,search_words
0,bresidendi,compound,"{presidendi, president}"
1,bresidendi,suggestion,"{presidendi, president}"


In [153]:
for idx, row in query_data.iterrows():
    for row['search_words']

SyntaxError: invalid syntax (739103293.py, line 2)

In [106]:
input_words = ['presidendi', 'ametiraha']

In [107]:
list( map(lambda x: x.lower(), input_words))

['presidendi', 'ametiraha']

In [ ]:
display(generate_all_worldforms('Tere').head())

In [ ]:
"""
curl --silent --request POST --header "Content-Type: application/json" \
        https://smart-search.tartunlp.ai/api/ea_paring_sl/version     
{"DB_LEMATISEERIJA":"./1024-koond.sqlite","version":"2023.10.14"}
(rt-web-crawler) swen@MacBook-Pro-6 riigi_teataja_pealkirjaotsing %
curl --silent --request POST --header "Content-Type: application/json" \
        --data '{"params":{"otsi_liitsõnadest":false}, "tss":"presidendiga\tpalk\tbresident"}' \
        https://smart-search.tartunlp.ai/api/ea_paring_sl/tsv
"""